In [1]:
from mxnet import nd
from mxnet.gluon import nn

def conv_block(channels):
    out = nn.Sequential()
    out.add(
        nn.BatchNorm(),
        nn.Activation('relu'),
        nn.Conv2D(4 * channels, kernel_size = 1),
        nn.BatchNorm(), 
        nn.Activation('relu'), 
        nn.Conv2D(channels, kernel_size = 3, padding = 1)
    )
    return out

class DenseBlock(nn.Block):
    def __init__(self, layers, growth_rate, **kwargs):
        super(DenseBlock, self).__init__(**kwargs)
        self.net = nn.Sequential()
        for i in range(layers):
            self.net.add(conv_block(growth_rate))
            
    def forward(self, x):
        for layer in self.net:
            out = layer(x)
            x = nd.concat(x, out, dim = 1)
        return x

/home/dyjng/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
dblk = DenseBlock(2, 10)
dblk.initialize()

x = nd.random.uniform(shape = (4, 3, 8, 8))
dblk(x).shape

(4, 23, 8, 8)

In [3]:
def transition_block(channels):
    out = nn.Sequential()
    out.add(
        nn.BatchNorm(), 
        nn.Activation('relu'), 
        nn.Conv2D(channels, kernel_size = 1), 
        nn.AvgPool2D(pool_size = 2, strides = 2)
    )
    return out

In [4]:
tblk = transition_block(10)
tblk.initialize()

tblk(x).shape

(4, 10, 4, 4)

In [5]:
init_channels = 64
growth_rate = 32    # k = 32
block_layers = [6, 12, 24, 16]
num_classes = 10

def dense_net():
    net = nn.Sequential()
    # add name_scope on the outermost Sequential
    with net.name_scope():
        # first block
        net.add(
            nn.Conv2D(init_channels, kernel_size = 7, strides = 2, padding = 3),  
            nn.BatchNorm(), 
            nn.Activation('relu'),
            nn.MaxPool2D(pool_size = 3, strides = 2, padding = 1)
        )
        # dense blocks
        channels = init_channels
        for i, layers in enumerate(block_layers):
            net.add(DenseBlock(layers, growth_rate))
            channels += layers * growth_rate         # m 
            if i != len(block_layers) - 1:
                net.add(transition_block(channels // 2))   # theta = 0.5  reduce theta * m
        # last block
        net.add(
            nn.BatchNorm(), 
            nn.Activation('relu'), 
            nn.AvgPool2D(pool_size = 1), 
            nn.Flatten(),  
            nn.Dense(num_classes)
        )
    return net

In [6]:
import sys
sys.path.append('..')
import utils
from mxnet import gluon
from mxnet import init

train_data, test_data = utils.load_data_fashion_mnist(
    batch_size = 64, resize = 32)

ctx = utils.try_gpu()
net = dense_net()
net.initialize(ctx = ctx, init = init.Xavier())

loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 
                        'sgd', {'learning_rate': 0.1})
utils.train(train_data, test_data, net, loss, 
            trainer, ctx, num_epochs = 5)

Start training on  gpu(0)
Epoch 0. Loss: 0.510, Train acc 0.82, Test acc 0.87, Time 84.9 sec
Epoch 1. Loss: 0.320, Train acc 0.88, Test acc 0.88, Time 84.2 sec
Epoch 2. Loss: 0.283, Train acc 0.89, Test acc 0.88, Time 84.4 sec
Epoch 3. Loss: 0.246, Train acc 0.91, Test acc 0.90, Time 84.1 sec
Epoch 4. Loss: 0.216, Train acc 0.92, Test acc 0.89, Time 84.0 sec
